In [ ]:
# ============================
# ECG Prediction Notebook
# ============================

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib  # to load saved scaler

# ----------------------------
# Step 1: Load the trained model
# ----------------------------
model = load_model("ecg_cnn_model.keras")

# ----------------------------
# Step 2: Load the saved scaler (same one used during training)
# ----------------------------
scaler = joblib.load("scaler.pkl")  # previously saved using joblib.dump

# ----------------------------
# Step 3: Load new ECG test data
# ============================
# ECG Test Script
# ============================

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib  # for loading saved scaler

# ----------------------------
# Step 1: Load trained model
# ----------------------------
model = load_model("ecg_cnn_model.keras")

# ----------------------------
# Step 2: Load saved scaler
# ----------------------------
scaler = joblib.load("scaler.pkl")  # saved after training

# ----------------------------
# Step 3: Load new ECG test data
# ----------------------------
new_df = pd.read_csv(r"data_sets\ecg_test.csv", header=None, dtype=str)

processed_signals = []
target_len = 187  # same as used in training

# ----------------------------
# Step 4: Preprocess each row
# ----------------------------
for idx, row in new_df.iterrows():
    # Convert to numeric, replace bad/malformed values with 0
    signal = pd.to_numeric(row, errors='coerce').fillna(0).values.flatten()
    
    # Pad or truncate to target length
    if len(signal) < target_len:
        signal = np.pad(signal, (0, target_len - len(signal)), mode='constant')
    elif len(signal) > target_len:
        signal = signal[:target_len]
    
    processed_signals.append(signal)

# Stack all signals for CNN input
new_X = np.stack(processed_signals)

# ----------------------------
# Step 5: Normalize using saved scaler
# ----------------------------
new_X = scaler.transform(new_X)

# Reshape for CNN input
new_X = new_X[..., np.newaxis]  # shape: (samples, signal_length, 1)

# ----------------------------
# Step 6: Predict
# ----------------------------
predictions = model.predict(new_X)
predicted_labels = (predictions > 0.5).astype(int)

# ----------------------------
# Step 7: Show predictions
# ----------------------------
results = []
for i, p in enumerate(predicted_labels):
    label = 'Abnormal' if p[0] == 1 else 'Normal'
    print(f"ECG {i+1}: {label}")
    results.append([i+1, label])

# ----------------------------
# Step 8: Save predictions to CSV
# ----------------------------
results_df = pd.DataFrame(results, columns=['ECG_ID', 'Prediction'])
results_df.to_csv("ecg_test_predictions.csv", index=False)
print("Predictions saved to 'ecg_test_predictions.csv'.")



FileNotFoundError: [Errno 2] No such file or directory: 'scaler.pkl'